In [1]:
# set sealing
# set tree_cover
# subdivision in hydrogeological regions_hg
# another subdivision into arbitrary regions_fokus (shapefile e.g. form QGIS)

<h1> Parameter </h1>

In [2]:
regions = [
    'rhine_graben_slab',
    'rhine_graben_zwischenscholle',
    'mainz_basin_tertiary',
    'quaternary_north_west',
    'quaternary_middle_rhine',
    'tertiary_quaternary_rhine_main',
    'palatinate_buntsandstein',
    'palatinate_muschelkalk',
    'triassic_north_west',
    'tertiary_north_west',    
    'palatinate_permokarbon',
    'perm_nahe_prims_basin',
    'palatinate_slate_mountain_range_south',
    'palatinate_slate_mountain_range_north',
    'lahn_dill',
    'westerwald_tertiary',
    'limestone_depression_buntsandstein_vulkanite',
    'kaenozoic_vulkanite'    ''
]
regions_fokus = ['RLP_total']

radius_list = [#50, 
    100 #, 150, 200, 250, 300
             ]

regions_path = '/home/jens/daten/geometries/selected_regions/RLP/'
regions_hg_path = '/home/jens/daten/geometries/geological_regions/RLP/hydrogeology'

file_folder="/home/jens/daten/RLP_temperatureLogs_shallow/T-Daten_RLP"
land_use_path = "/home/jens/daten/parameter/lbm-de2021.utm32s.shape/lbm-de2021/land/rp/lbm-de2021.shp" # EPSG:25832
tree_cover_path = "/home/jens/daten/parameter/TCD_2012_100m_eu_03035_d04_Full/TCD_2012_100m_eu_03035_d04_full.tif"

<h1> Import </h1>

In [3]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from geoalchemy2 import Geometry

from scipy.spatial import cKDTree
from scipy.ndimage import sobel
from scipy.stats import linregress, ks_2samp, wasserstein_distance

import geopandas as gpd
from shapely.geometry import Point
from pyproj import Transformer

import rioxarray
import xarray as xr

import rasterio
from rasterio.transform import rowcol
from rasterio.enums import Resampling
from rasterio.warp import transform, reproject, calculate_default_transform, Resampling
from rasterstats import zonal_stats
from rasterio.shutil import copy as rio_copy
from rasterio.crs import CRS

import seaborn as sns

import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator

from math import pi
import pprint
from tabulate import tabulate

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


In [4]:
user, password ='postgres', 'postgres'
schema, host = 'DB_RLP_temperature_shallow', '127.0.0.1'
connection_string='postgresql://{}:{}@{}:5432/{}'.format(user, password, host, schema)

engine = create_engine(connection_string)

df_fitting = pd.read_sql_query("SELECT * FROM fitting;", engine)
df_borehole_evaluation = pd.read_sql_query("SELECT * FROM borehole_evaluation;", engine)
df_temperatures = pd.read_sql_query("SELECT * FROM temperatures;", engine)
df_infos = pd.read_sql_query("SELECT * FROM log_infos;", engine)
df_infos_unique = df_infos.drop_duplicates(subset='borehole_id', keep='first')



df_temperatures.loc[df_temperatures['T_0'] > 1e6, 'T_0'] = np.nan
df = pd.merge(df_temperatures, df_infos_unique[['borehole_id', #'T_surface_modis',
                                                'elevation', 
                                                'sealing', 'vegetation',
                                               'land_use', 'land_cover', 'tree_cover',
                                                'utm_easting', 'utm_northing']], 
              on='borehole_id', how='left')

#df = pd.merge(df, df_borehole_evaluation[['borehole_id', 'selected_sample']], on='borehole_id', how='left')
df = pd.merge(df, df_fitting[['borehole_id', 'points', 'number_of_points']], on='borehole_id', how='left')
df = pd.merge(df, df_borehole_evaluation[['borehole_id', 'drillingInduced', 'deactivated', 'quality']], on='borehole_id', how='left')

def clean_and_convert(points_str):
    cleaned = points_str.replace('\n', '').replace('[', '').replace(']', '').strip()
    return np.fromstring(cleaned, sep=' ').reshape(-1, 2)  # -1 passt automatisch die Zeilen an

# Neue Spalte mit NumPy-Arrays
df['points_array'] = df['points'].apply(clean_and_convert)


#df.columns# head()
df['region_hg'] = pd.NA

<h1> Processing </h1>

In [5]:
def calculate_sealing(df_points, gdf_versiegelung, radius_dict, value_column='SIE_AKT'):
    """
    df_points: pandas.DataFrame mit 'utm_easting' und 'utm_northing'
    gdf_versiegelung: GeoDataFrame mit Flächenversiegelung (Polygon), CRS = EPSG:32632
    radius_dict: dict → {'sealing_50': 28.2, 'sealing_100': 56.4, ...}
    value_column: Spalte mit Versiegelungsgrad (z. B. 'SIE_AKT')
    """

    # 1. GeoDataFrame aus Punkten erzeugen
    gdf_points = gpd.GeoDataFrame(
        df_points.copy(),
        geometry=gpd.points_from_xy(df_points['utm_easting'], df_points['utm_northing']),
        crs="EPSG:32632"
    )

    print("CRS Punkte:", gdf_points.crs)
    print("CRS Versiegelung:", gdf_versiegelung.crs)

    # 2. Über jeden gewünschten Radius iterieren
    for colname, radius in radius_dict.items():
        print(f"\nRadius {radius:.1f} m → Spalte '{colname}'")

        # a) Buffer um Punkte
        gdf_buffer = gdf_points.copy()
        gdf_buffer['geometry'] = gdf_buffer.geometry.buffer(radius)

        # b) Original-Index sichern für späteres Gruppieren
        gdf_buffer['orig_index'] = gdf_buffer.index

        # c) Overlay (Schnittmenge mit Versiegelungsflächen)
        intersection = gpd.overlay(gdf_buffer, gdf_versiegelung, how='intersection')

        print(f"Overlay-Ergebnis: {len(intersection)} Zeilen")

        if intersection.empty:
            print("Kein Treffer – alle Werte werden auf 0 gesetzt")
            df_points[colname] = 0.0
            continue

        # d) Flächeninhalt berechnen
        intersection['area'] = intersection.geometry.area

        # e) Gewichteter Mittelwert: Versiegelung * Fläche
        def weighted_sealing(gr):
            total_area = gr['area'].sum()
            if total_area == 0:
                return 0.0
            return (gr[value_column] * gr['area']).sum() / total_area

        # f) Gruppieren nach Punkt (originaler Index)
        sealing_series = intersection.groupby('orig_index', group_keys=False, observed=False, sort=False).apply(weighted_sealing)


        # g) Ergebnis-Spalte zurück ins DataFrame
        df_points[colname] = sealing_series.reindex(df_points.index, fill_value=np.nan)

    return df_points

# changed from area to radius

radius_dict = {}
for radius in  radius_list:
    radius_dict[f'sealing_{radius}'] = radius
    

gdf_versiegelung = gpd.read_file(land_use_path)
gdf_versiegelung = gdf_versiegelung.to_crs("EPSG:32632")

df = calculate_sealing(df, gdf_versiegelung, radius_dict)

CRS Punkte: EPSG:32632
CRS Versiegelung: EPSG:32632

Radius 100.0 m → Spalte 'sealing_100'
Overlay-Ergebnis: 2253 Zeilen


/tmp/ipykernel_10451/4269318463.py:51: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sealing_series = intersection.groupby('orig_index', group_keys=False, observed=False, sort=False).apply(weighted_sealing)


In [6]:
def treecover_in_buffers(df_points, raster_path, _radius_list, prefix="tree_cover"):
    """
    Berechnet den mittleren Tree-Cover-Wert im Umkreis (Buffer) um Punkte.
    
    Parameter
    ----------
    df_points : GeoDataFrame oder DataFrame
        Punktdaten mit Spalten 'utm_easting' und 'utm_northing' (Meter).
    raster_path : str
        Pfad zum Tree-Cover Raster (GeoTIFF).
    radii_m : list[int]
        Liste von Buffer-Radien in Metern (z. B. [50, 100, 200]).
    prefix : str
        Präfix für die Ergebnisspalten (Standard: "tree_cover").
        
    Rückgabe
    --------
    GeoDataFrame mit allen ursprünglichen Spalten und neuen Spalten wie
    tree_cover_50, tree_cover_100, ...
    """

    gdf_points = gpd.GeoDataFrame(
        df_points.copy(),
        geometry=gpd.points_from_xy(df_points['utm_easting'], df_points['utm_northing']),
        crs="EPSG:32632"
    )

    with rasterio.open(raster_path) as src:
        raster_crs = src.crs
        nodata = src.nodata

    gdf_points = gdf_points.to_crs(raster_crs)
    results = gdf_points.copy()

    # Berechnung pro Buffer-Radius
    with rasterio.open(raster_path) as src:
        for r in _radius_list:
            #r = np.sqrt(a_sqrt*a_sqrt / np.pi)
            print(f'treecover_{r} - Radius: {r}')
            values = []
            for i, geom in enumerate(gdf_points.geometry):
                try:
                    buffer = geom.buffer(r)
                    out_image, _ = rasterio.mask.mask(src, [buffer], crop=True)
                    data = out_image[0]
                    if nodata is not None:
                        data = data[data != nodata]
                    data = data[~np.isnan(data)]
                    mean_val = np.mean(data) if data.size > 0 else np.nan
                except Exception:
                    mean_val = np.nan
                values.append(mean_val)

            results[f"{prefix}_{r}"] = values

    results = results.to_crs("EPSG:32632")

    return results

df = treecover_in_buffers(df, tree_cover_path, radius_list)

treecover_100 - Radius: 100


In [7]:
for region in regions:
    shapefile = 'region_{}.shp'.format(region)
    
    polygon_gdf = gpd.read_file(regions_hg_path + '/' + shapefile) 
    polygon_gdf = polygon_gdf.to_crs(epsg=25832)

    # boreholes
    geometry = [Point(xy) for xy in zip(df["utm_easting"], df["utm_northing"])]
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:25832")
    gdf_filtered = gdf[gdf.within(polygon_gdf.geometry.union_all())]

    df.loc[gdf_filtered.index, 'region_hg'] = region

In [8]:
for region in regions_fokus:
    df[f'region_{region}'] = False
    
    shapefile = 'region_{}.shp'.format(region)
    
    polygon_gdf = gpd.read_file(regions_path + '/' + shapefile)
    polygon_gdf = polygon_gdf.to_crs(epsg=25832) 
    
    # boreholes
    geometry = [Point(xy) for xy in zip(df["utm_easting"], df["utm_northing"])]
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:25832")  # Falls UTM-Zone 33N
    gdf_filtered = gdf[gdf.within(polygon_gdf.geometry.union_all())]
    df.loc[gdf_filtered.index, f'region_{region}'] = True
    
# df.head()

In [9]:
df.to_csv('df4statstools.csv')

In [10]:
df[df['sealing_100'].isna()]

,borehole_id,geometry,T_surface_landsat,T_surface_modis,T_0,T_10,T_20,T_30,T_40,T_50,...,points,number_of_points,drillingInduced,deactivated,quality,points_array,region_hg,sealing_100,tree_cover_100,region_RLP_total
450,GT_4954,POINT (458009.909 5489426.69),3.400000e+38,0.0,None,NaN,NaN,NaN,NaN,NaN,...,[[ 61.7 12.04]\n [1000. 41.48]\n [ 601...,6,False,False,0,"[[61.7, 12.04], [1000.0, 41.48], [601.0, 26.27...",rhine_graben_zwischenscholle,NaN,6.444444,True
